<a href="https://colab.research.google.com/github/SimonEngler/nearestNeighbor/blob/master/engler_submit_hw1_28Jan2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can do this homework online through [Google Colab](https://colab.research.google.com/). By linking Colab to Github, you can import this file to colab, and save your changes back to Github directly.


# Homework 1: Naive Bayes Classifier

In this homework you will implement the Naive Bayes Classsifier on a data set of votes in the U.S. House of Representatives, with the goal of predicting the party affiliation of each congressman. The input data $X$ is given by a $N$-by-$D$ matrix, where $N$ is the number of examples and $D=16$ is the number of input features. Each feature is binary (yes/no). The targets are given by a length-$N$ sequence of classes, $Y$, that are also binary. More information on the data set can be found at  https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records.



First, we need to download the data. The following code uses the urllib library to request data from a website. The pandas library is a powerful library for data analysis --- we use the read_csv method to automatically parse the comma seperated variable (csv) file.

In [1]:
import pandas 
import urllib.request  
import numpy   # Numerical python.
import math
import collections

# Download the data.
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data"
response = urllib.request.urlopen(url)

# Interpret text data into pandas data frame. Interpret 'abstain' votes as 'no'.
dataset  = pandas.read_csv(response, header=None, true_values=['y'], false_values=['n','?'])

# Set the column names.
names = ['label'] + [f'vote_{i}' for i in range(16)]
dataset.columns = names

# Tells pandas that this is a categorical feature.
dataset['label'] = pandas.Categorical(dataset['label'])

print("Dataset shape: ", dataset.shape)
dataset.head() # Prints first 5 examples from the data set.

Dataset shape:  (435, 17)


,label,vote_0,vote_1,vote_2,vote_3,vote_4,vote_5,vote_6,vote_7,vote_8,vote_9,vote_10,vote_11,vote_12,vote_13,vote_14,vote_15
0,republican,False,True,False,True,True,True,False,False,False,True,False,True,True,True,False,True
1,republican,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False
2,democrat,False,True,True,False,True,True,False,False,False,False,True,False,True,True,False,False
3,democrat,False,True,True,False,False,True,False,False,False,False,True,False,True,False,False,True
4,democrat,True,True,True,False,True,True,False,False,False,False,True,False,True,True,True,True


Numpy is a powerful library for mathematical operations on vectors and matrices. Here we convert the pandas data into a 2-dimensional numpy array (a matrix). 

In [2]:
X = numpy.array(dataset.iloc[:,1:]) # Convert input features into Numpy array.
Y = dataset['label'].cat.codes # Converts string labels to binary values.

# Split data into train and test set. Use the first 335 examples for training.
num_train = 335
Xtrain = X[0:num_train, :].astype('float32')
Ytrain = Y[0:num_train].astype('f')
Xtest  = X[num_train:,:].astype('f')
Ytest  = Y[num_train:].astype('f')

print(Xtrain.shape, Xtest.shape)

(335, 16) (100, 16)


You are asked to implement the following functions.

In [3]:
def get_distance(q, X):
  #Distance between query q and every point in data matrix X
  np = numpy
  N, D = X.shape
  assert(len(q) == D)
  rval = np.zeros((N,))
  for i in range(N):
    distance = np.sqrt(np.sum((q - X[i, :])**2))
    rval[i] = distance
  return rval

def get_distance_vectorized(q,X):
  np = numpy
  return np.sqrt(np.sum((q - X)**2, axis=1)) #broadcasting vector into matrix

get_distance(Xtest[0,:], Xtrain[:10,:])

get_distance_vectorized(Xtest[0,:], Xtrain[:10,:])



array([1.7320508, 1.7320508, 2.236068 , 2.828427 , 2.828427 , 2.828427 ,
       2.6457512, 2.236068 , 2.       , 3.7416575], dtype=float32)

In [4]:
def classify(q, Xtrain, Ytrain):
  np = numpy
  distances = get_distance_vectorized(q, Xtrain)
  min_idx = np.argmin(distances)
  #This is the minimum value: distances[min_idx]
  return Ytrain[min_idx]

np = numpy
preds = np.zeros((100,))
for i,q in enumerate(Xtest):
  preds[i] = classify(q, Xtrain, Ytrain)
print("Accuracy is:%0.2f" % (np.sum(preds == Ytest)/len(Ytest)))
 

Accuracy is:0.87


In [0]:

#Reference: (https://hackernoon.com/implementation-of-gaussian-naive-bayes-in-python-from-scratch-c4ea64e3944d)
def generative_model(Xtrain, Ytrain):
    ''' i
    Implements a generative algorithm on binary data.
    Inputs
        Xtrain: NxD matrix of features.
        Ytrain: N vector of class labels
    
    Returns
        p_label: Length 2 vector of class probabilities.
        p_votes: 2xD Matrix where entry i,j is p(x_j|v=i).
    ''' 
    # WRITE ME
    p_label = np.ones(2)
    y_dict = collections.Counter(Ytrain)
    for i in range(0,2):
      p_label[i] = y_dict[i]/Ytrain.shape[0]
       
    #p_label = [0.5, 0.5]   
    #Probability of voting using Xtrain votes
    D = Xtrain.shape[1]
    N = Xtrain.shape[0]
    p_votes = np.ones((2,D))
    total_R = np.ones(D)
    total_D = np.ones(D)
    
    #Count number of Republicans
    num_republicans = 0
    num_democrats = 0
    for i in range(0,N):
        if Ytrain[i] == 1:
            num_republicans = num_republicans + 1
        else:
            num_democrats = num_democrats + 1
    print("Republicans: ", num_republicans)
    print("Democrats: ", num_democrats)
    
    #Split republican and democrat votes
    D_prob = np.ones((num_democrats,D))
    R_prob = np.ones((num_republicans,D))
     
    #If republican
    for i in range(0, num_republicans):
        #if Republican
        if Ytrain[i] == 1:
            R_prob[i][:] = Xtrain[i][:]
    print("R_prob", R_prob)
    #democrat
    for i in range(0, num_democrats):
        if Ytrain[i] == 0:
            D_prob[i][:] = Xtrain[i][:]
    print("D_prob", D_prob)        
    for i in range(0, D):
        total_D[i] = sum(D_prob[:][i])
        p_votes[0][i] = (total_D[i]/Xtrain.shape[1])*p_label[0]
    print("TOTAL_D:", total_D)
    for i in range(0, D):
        total_R[i] = sum(R_prob[:][i])
        p_votes[1][i] = (total_R[i]/Xtrain.shape[1])*p_label[1]
    
    print("TOTAL_R", total_R)
    #Calculate 
    print("Labels: ", p_label)
    print("Votes: ", p_votes)
    
     #Calculate Log-Likelyhood
    Loglike = 0
    for i in range(0,p_votes.shape[0]):
        for j in range(0,p_votes.shape[1]):
            Loglike = Loglike + np.log((p_label[i])) + np.log((p_votes[i][j]))*np.log((p_label[i]))
        
    print("Log Sum:", Loglike)
    
    return p_label, p_votes
  
def discriminative_model(p_label, p_votes, Xtest):
    '''
    Implements Naive Bayes Classification.
    Inputs
      p_label, p_votes: From generative_model.
      Xtest: NxD matrix of binary features.
    
    Outputs 
      pred_prob: Probability of label=1 under model.
    '''
    # WRITE ME
    N, D = Xtest.shape
    pfc = np.ones(2)
    pcf = np.ones(2)

    for i in range(0, 2):
        product = 1
        for j in range(0, D):
           # if(Xtest.shape[0] == 1):
           #     product = p_votes[i][j] * Xtest[0][j]
           # else:
           product = p_votes[i][j] * Xtest[i][j]
        pfc[i] = product
    
    total_prob = 0
    for i in range(0, 2):
        total_prob = total_prob + (pfc[i] * p_label[i])
    #print("Total Prob:", total_prob)
    for i in range(0, 2):
        pcf[i] = (pfc[i] * p_label[i])/total_prob
  #  print("PCF:", pcf)
    pred_prob = pcf[1]
    return pred_prob

def accuracy(y_true, y_predicted):
    ''' Calculates the fraction of con\rrect predictions.
    '''
    #assert(len(y_true) == len(y_predicted))
    # WRITE ME
    count = 0
    for i in range(0,len(y_true)):
        if y_predicted == y_true[i]:
            count = count + 1
        
    fraction = 1 - count/len(y_true)
    print("Total Accuracy: ", fraction)
    
    

In [16]:
# Make sure to print these for submission.
p_label, p_votes = generative_model(Xtrain, Ytrain)
print('Label priors:', p_label)
print('Conditional vote probabilities:', p_votes)

pred_prob = discriminative_model(p_label, p_votes, Xtest)
print('Predictions:', pred_prob)


#The log probability of the test set is calculated as:


Republicans:  127
Democrats:  208
R_prob [[0. 1. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]]
D_prob [[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [0. 1. 1. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
TOTAL_D: [16. 16.  7.  6. 10.  8.  7. 16. 16.  6. 16. 16.  6.  9. 16. 16.]
TOTAL_R [ 9.  7. 16. 16. 16. 16. 16.  7.  8. 16.  5.  7. 16. 16.  5.  7.]
Labels:  [0.62089552 0.37910448]
Votes:  [[0.62089552 0.62089552 0.27164179 0.23283582 0.3880597  0.31044776
  0.27164179 0.62089552 0.62089552 0.23283582 0.62089552 0.62089552
  0.23283582 0.34925373 0.62089552 0.62089552]
 [0.21324627 0.16585821 0.37910448 0.37910448 0.37910448 0.37910448
  0.37910448 0.16585821 0.18955224 0.37910448 0.11847015 0.16585821
  0.37910448 0.37910448 0.11847015 0.16585821]]
Log Sum: 5.255315287948211
Label priors: [0.62089552 0.37910448]
Conditional vote p

## To turn in:
1) Implement the Naive Bayes Classifier using the starter code above. Make sure to print out the parameters of the generative model and the predictions on the test points.

**See code above.**

Republicans:  127
Democrats:  208
R_prob [[0. 1. 0. ... 1. 0. 1.]
 [0. 1. 0. ... 1. 0. 0.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]]
D_prob [[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [0. 1. 1. ... 1. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
TOTAL_D: [16. 16.  7.  6. 10.  8.  7. 16. 16.  6. 16. 16.  6.  9. 16. 16.]
TOTAL_R [ 9.  7. 16. 16. 16. 16. 16.  7.  8. 16.  5.  7. 16. 16.  5.  7.]
Labels:  [0.62089552 0.37910448]
Votes:  [[0.62089552 0.62089552 0.27164179 0.23283582 0.3880597  0.31044776
  0.27164179 0.62089552 0.62089552 0.23283582 0.62089552 0.62089552
  0.23283582 0.34925373 0.62089552 0.62089552]
 [0.21324627 0.16585821 0.37910448 0.37910448 0.37910448 0.37910448
  0.37910448 0.16585821 0.18955224 0.37910448 0.11847015 0.16585821
  0.37910448 0.37910448 0.11847015 0.16585821]]
Label priors: [0.62089552 0.37910448]
Conditional vote probabilities: [[0.62089552 0.62089552 0.27164179 0.23283582 0.3880597  0.31044776
  0.27164179 0.62089552 0.62089552 0.23283582 0.62089552 0.62089552
  0.23283582 0.34925373 0.62089552 0.62089552]
 [0.21324627 0.16585821 0.37910448 0.37910448 0.37910448 0.37910448
  0.37910448 0.16585821 0.18955224 0.37910448 0.11847015 0.16585821
  0.37910448 0.37910448 0.11847015 0.16585821]]
Predictions: 1.0


2) Compute the log probability of the test set --- this is a single scalar value.

Log Probability: 5.255315287948211

3) Compare the NB classifier to a model in which we predict a 50-50 chance for each vote, in terms of accuracy and the log probability. Which model is better and why? Describe two situations in which the Naive Bayes Classifier will fail. 


**The log probability is better as it allows you to find the maximum log probability. The 50-50 chance for each vote does not predict well using the training data. ** 

** A Naive Bayes classifyer will fail if the only seperation between the two classes is the covariance. Also, it due to the conditional independance if you have no occurances of a catagory in your sample it can drop your probability estimate to zero      **

**References**
**https://www.youtube.com/watch?v=feBKiAdhYkc**
**https://www.researchgate.net/post/What_are_the_disadvantages_of_Naive_Bayes**


